# Higher Order SVD (HOSVD) and TRG (HOTRG)

## References
* PRB **86**, 045139 (2012).

## Higer Order SVD (HOSVD)

Given a rank-4 tensor $M_{abcd}$, we want to find unitary matrices $U^L, U^R, U^U, U^D$ such that
$$
  M_{abcd} = \sum_{ijkl} S_{ijkl} U^L_{ai} U^R_{bj} U^U_{ck} U^{D}_{dl},
$$
where $S$ is the core tensor.

In [1]:
import Tor10
import copy
import numpy as np

def Tprint(T):
    print(T.Print_diagram())
    print(T)
    
def Tprint_diag(T):
    diag = np.array([T[i,i].item() for i in range(T.bonds[0].dim)])
    print(diag)

In [3]:
bd = Tor10.Bond(2)
M = Tor10.UniTensor([bd, bd, bd, bd], rowrank=2, name='M')
M.Rand()
# Tprint(M)
M.Print_diagram()

-----------------------
tensor Name : M
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 3  
           \             /     
            -------------      


### How to evaluate $U^L$

* Define rank-2 tensor $ML_{a,bcd} = M_{abcd}$.
* Define rank-2 tensor $[MM]=[ML][ML]^T$.

In [4]:
ML = M.Permute([0,1,2,3], rowrank=1)
ML.SetName('ML')
ML_t = M.Whole_transpose()
ML_t.SetName('ML_t')

ML.SetLabel(10, 0)
ML_t.SetLabel(11, 3)

# Tprint(ML)
ML.Print_diagram()
ML_t.Print_diagram()

-----------------------
tensor Name : ML
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    10 ____| 2         2 |____ 1  
           |             |     
           |           2 |____ 2  
           |             |     
           |           2 |____ 3  
           \             /     
            -------------      
-----------------------
tensor Name : ML_t
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     1 ____| 2         2 |____ 11 
           |             |     
     2 ____| 2           |        
           |             |     
     3 ____| 2           |        
           \             /     
            -------------      


In [5]:
X = Tor10.Contract(ML, ML_t)
# X.Print_diagram()
Tprint(X)

-----------------------
tensor Name : 
tensor Rank : 2
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    10 ____| 2         2 |____ 11 
           \             /     
            -------------      
None
Tensor name: 
is_diag    : False
tensor([[3.3867, 2.2059],
        [2.2059, 2.2591]], dtype=torch.float64)



In [6]:
U, _, _ = X.Svd()
Ut = U.Whole_transpose()
Ut.SetLabels([-1, 11])
Tprint(U)
Tprint(Ut)

# check unitary 
Tprint(Tor10.Contract(Ut, U))
Tprint(Tor10.Contract(U, Ut))

-----------------------
tensor Name : 
tensor Rank : 2
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    10 ____| 2         2 |____ -1 
           \             /     
            -------------      
None
Tensor name: 
is_diag    : False
tensor([[-0.7898, -0.6133],
        [-0.6133,  0.7898]], dtype=torch.float64)

-----------------------
tensor Name : 
tensor Rank : 2
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    -1 ____| 2         2 |____ 11 
           \             /     
            -------------      
None
Tensor name: 
is_diag    : False
tensor([[-0.7898, -0.6133],
        [-0.6133,  0.7898]], dtype=torch.float64)

-----------------------
tensor Name : 
tensor Rank : 2
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    10 ____| 2         2 